In [1]:
import os

export_root = '/data2/chence/ExportedMVhq'
json_files = sorted([i for i in os.listdir(export_root) if i.endswith('.json') and i.startswith('meta_')])

In [2]:
import json

merged_data = {}
for idx in range(len(json_files)):
    json_file = json_files[idx]
    print(idx, json_file)
    with open(os.path.join(export_root, json_file), 'r') as f:
        data = json.load(f)
        merged_data.update(data)
print(len(merged_data))

0 meta_0000000.json
1 meta_0011931.json
2 meta_0121747.json
268807


In [3]:
import os.path as osp
import tqdm

del_keys = []

for k in tqdm.tqdm(merged_data.keys()):
    align_image_file = osp.join(export_root, 'align_images', k)
    align_parsing_file = osp.join(export_root, 'align_parsing', k)
    head_image_file = osp.join(export_root, 'head_images', k)
    head_parsing_file = osp.join(export_root, 'head_parsing', k)
    if not osp.exists(align_image_file) or not osp.exists(align_parsing_file) or not osp.exists(head_image_file) or not osp.exists(head_parsing_file):
        del_keys.append(k)

100%|██████████| 268807/268807 [00:13<00:00, 19957.51it/s]


In [4]:
len(del_keys)

0

In [5]:
len(merged_data)

268807

In [6]:
for k in del_keys:
    del merged_data[k]

In [7]:
len(merged_data)

268807

In [8]:
exportjpg_root = '/data2/chence/multi_view_hq'
os.makedirs(exportjpg_root, exist_ok=True)

with open(os.path.join(exportjpg_root, 'dataset.json'), 'w') as f:
    json.dump(merged_data, f)

In [9]:
os.system('cp -r %s %s' % (os.path.join(export_root, 'align_parsing'), exportjpg_root))

0

In [10]:
os.system('cp -r %s %s' % (os.path.join(export_root, 'head_parsing'), exportjpg_root))

0

In [9]:
import cv2
import os.path as osp

def work(image_name, export_root, exportjpg_root):
    try:
        base_folder, base_name = image_name.split('/')
        align_png_path = os.path.join(export_root, 'align_images', base_folder, base_name)
        assert os.path.exists(align_png_path)
        head_png_path = os.path.join(export_root, 'head_images', base_folder, base_name)
        assert os.path.exists(head_png_path)
        os.makedirs(os.path.join(exportjpg_root, 'align_images', base_folder), exist_ok=True)
        os.makedirs(os.path.join(exportjpg_root, 'head_images', base_folder), exist_ok=True)
        align_jpg_path = os.path.join(exportjpg_root, 'align_images', base_folder, base_name.replace('.png', '.jpg'))
        if not osp.exists(align_jpg_path):
            cv2.imwrite(align_jpg_path, cv2.imread(align_png_path))
        head_jpg_path = os.path.join(exportjpg_root, 'head_images', base_folder, base_name.replace('.png', '.jpg'))
        if not osp.exists(head_jpg_path):
            cv2.imwrite(head_jpg_path, cv2.imread(head_png_path))
    except Exception as e:
        print(e)

In [10]:
image_names = sorted(merged_data.keys())

In [11]:
import multiprocessing as mp
from tqdm import tqdm

global progress_bar
progress_bar = tqdm(total=len(image_names))

def update_progress_bar(_):
    progress_bar.update()

pool = mp.Pool(processes=64)
for image_name in image_names:
    pool.apply_async(work, (
        image_name,
        export_root,
        exportjpg_root,
    ), callback=update_progress_bar)
pool.close()
pool.join()

100%|██████████| 268807/268807 [3:53:39<00:00, 29.98it/s]   

100%|██████████| 268807/268807 [3:53:58<00:00, 29.98it/s]